# Часть 1

Предполагается, что у вас есть следующие таблицы:

Таблица employees:
- id (идентификатор сотрудника)
- name (имя сотрудника)
- department_id (идентификатор отдела)
- salary (зарплата сотрудника)

Таблица departments:
- id (идентификатор отдела)
- name (название отдела)

Таблица sales:
- id (идентификатор продажи)
- employee_id (идентификатор сотрудника)
- sale_date (дата продажи)
- amount (сумма продажи)

In [1]:
import sqlite3
from faker import Faker
import random
import datetime
import pandas as pd

conn = sqlite3.connect('company.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                (id INTEGER PRIMARY KEY, name TEXT, department_id INTEGER, salary INTEGER)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS departments
                (id INTEGER PRIMARY KEY, name TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS sales
                (id INTEGER PRIMARY KEY, employee_id INTEGER, sale_date DATE, amount INTEGER)''')

cursor.execute("SELECT COUNT(*) FROM departments")
count_departments = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM employees")
count_employees = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM sales")
count_sales = cursor.fetchone()[0]

if count_departments == 0 and count_employees == 0 and count_sales == 0:
    fake = Faker()

    for _ in range(5):
        cursor.execute("INSERT INTO departments (name) VALUES (?)", (fake.job(),))
    department_ids = [row[0] for row in cursor.execute("SELECT id FROM departments")]
    for _ in range(20):
        name = fake.name()
        department_id = random.choice(department_ids)
        salary = random.randint(30000, 80000)
        cursor.execute("INSERT INTO employees (name, department_id, salary) VALUES (?, ?, ?)", (name, department_id, salary))

    employee_ids = [row[0] for row in cursor.execute("SELECT id FROM employees")]
    for _ in range(300):
        employee_id = random.choice(employee_ids)
        sale_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
        amount = random.randint(1000, 10000)
        cursor.execute("INSERT INTO sales (employee_id, sale_date, amount) VALUES (?, ?, ?)", (employee_id, sale_date, amount))

    conn.commit()
else:
    print("Данные уже заполнены")
#conn.close()

In [2]:
#cursor.execute('DELETE FROM employees')
#cursor.execute('DELETE FROM departments')
#cursor.execute('DELETE FROM sales')
#conn.commit()

In [3]:
print("Employees Table:")
pd.read_sql_query("SELECT * FROM employees", conn)

Employees Table:


,id,name,department_id,salary
0,1,Jennifer Hernandez,2,39227
1,2,Sandra Vasquez,4,71423
2,3,Ashley Douglas,1,78814
3,4,Danielle Barnes,1,71725
4,5,Heidi Moore,2,61299
5,6,Melissa Patel,5,42775
6,7,Brian Edwards,1,77957
7,8,Erik Henry,2,40413
8,9,Bryan Davis,1,48044
9,10,Carol Martin,5,50686


In [4]:
print("\nDepartments Table:")
pd.read_sql_query("SELECT * FROM departments", conn)


Departments Table:


,id,name
0,1,Private music teacher
1,2,Teaching laboratory technician
2,3,"Buyer, retail"
3,4,Broadcast presenter
4,5,Heritage manager


In [5]:
print("\nSales Table:")
pd.read_sql_query("SELECT * FROM sales ORDER BY sale_date", conn)


Sales Table:


,id,employee_id,sale_date,amount
0,3,13,2022-12-21,8244
1,72,20,2022-12-22,5964
2,5,3,2022-12-25,3336
3,47,20,2022-12-26,2809
4,211,14,2022-12-26,4081
...,...,...,...,...
295,146,4,2023-12-10,9911
296,104,8,2023-12-11,3686
297,197,11,2023-12-13,4367
298,296,19,2023-12-13,1831


1. Найти общую зарплату по каждому отделу.

In [6]:
pd.read_sql("""
    SELECT 
         d.name AS department
        ,SUM(e.salary) AS total_salary
    FROM employees e
    JOIN departments d ON e.department_id = d.id
    GROUP BY d.name
""", conn)

,department,total_salary
0,Broadcast presenter,150736
1,Heritage manager,135414
2,Private music teacher,444978
3,Teaching laboratory technician,316293


2. Найти общую сумму продаж по каждому сотруднику.

In [7]:
pd.read_sql("""
    SELECT 
         e.name AS employee
        ,SUM(s.amount) AS total_sales
    FROM sales s
    JOIN employees e ON s.employee_id = e.id
    GROUP BY e.name
""", conn)

,employee,total_sales
0,Angela Lee,82808
1,Ashley Douglas,103512
2,Brian Edwards,113698
3,Bryan Davis,47297
4,Carol Martin,82731
5,Corey Torres,53700
6,Danielle Barnes,99530
7,Erik Henry,80773
8,Heidi Moore,53237
9,Jennifer Hernandez,102838


3. Найти сумму продаж для каждого сотрудника по месяцам.

In [8]:
pd.read_sql("""
    SELECT 
         e.name AS employee
        ,strftime('%m', s.sale_date) AS month
        ,SUM(s.amount) AS total_sales
    FROM sales s
    JOIN employees e ON s.employee_id = e.id
    GROUP BY e.name, strftime('%m', s.sale_date)
""", conn)

,employee,month,total_sales
0,Angela Lee,01,5247
1,Angela Lee,02,2033
2,Angela Lee,03,6797
3,Angela Lee,04,5948
4,Angela Lee,05,16079
...,...,...,...
170,Taylor Crosby,06,1027
171,Taylor Crosby,09,15993
172,Taylor Crosby,10,9530
173,Taylor Crosby,11,13775


4. Найти среднюю зарплату ,  максимальную зарплату, минимальную зарплату по каждому отделу.

In [9]:
pd.read_sql("""
    SELECT 
         d.name AS department
        ,AVG(e.salary) AS average_salary
        ,MAX(e.salary) AS max_salary
        ,MIN(e.salary) AS min_salary
    FROM employees e
    JOIN departments d ON e.department_id = d.id
    GROUP BY d.name
""", conn)

,department,average_salary,max_salary,min_salary
0,Broadcast presenter,75368.000000,79313,71423
1,Heritage manager,45138.000000,50686,41953
2,Private music teacher,55622.250000,78814,38511
3,Teaching laboratory technician,45184.714286,63081,31270


5. Найти общую сумму продаж по дням недели, по кварталам, по годам.

In [10]:
# Тут я не очень понял, отдельные ли запросы нужны или всё вместе.
# Решил, что в статистике продаж по кварталам по дням неделе в сумме тоже имеет смысл и сделал второй вариант.

pd.read_sql("""
    SELECT
        CASE CAST(strftime('%w', s.sale_date) AS INTEGER)
            WHEN 0 THEN 'Sunday'
            WHEN 1 THEN 'Monday'
            WHEN 2 THEN 'Tuesday'
            WHEN 3 THEN 'Wednesday'
            WHEN 4 THEN 'Thursday'
            WHEN 5 THEN 'Friday'
            ELSE 'Saturday'
        END AS day_of_week,
        CAST(strftime('%m', s.sale_date)/4 + 1 AS INTEGER) AS quarter,
        CAST(strftime('%Y', s.sale_date) AS INTEGER) AS year,
        SUM(s.amount) AS total_sales
    FROM
        sales s
    GROUP BY
        day_of_week, quarter, year
    ORDER BY
        year, 
        quarter, 
        /* На российский манер, начиная с понедельника */
        CASE day_of_week
            WHEN 'Monday' THEN 1
            WHEN 'Tuesday' THEN 2
            WHEN 'Wednesday' THEN 3
            WHEN 'Thursday' THEN 4
            WHEN 'Friday' THEN 5
            WHEN 'Saturday' THEN 6
            ELSE 7
        END;
""", conn)

,day_of_week,quarter,year,total_sales
0,Monday,4,2022,6890
1,Wednesday,4,2022,11827
2,Thursday,4,2022,13903
3,Sunday,4,2022,3336
4,Monday,1,2023,90138
5,Tuesday,1,2023,90113
6,Wednesday,1,2023,33864
7,Thursday,1,2023,47786
8,Friday,1,2023,56009
9,Saturday,1,2023,75105


# Часть 2

Есть таблица t с транзакциями клиентов, начиная с 2021 года. В ней столбцы – 
id_client 
tran_time,
id_tran,
sum_tran,
prod_name.

id_client     tran_time    id_tran    sum_tran    prod_name
    38      01.04.2020      11          218.5    основной
    28      01.04.2020      12          230.0    дополнительный
    14      01.04.2020      13          117.0    основной
    8       01.04.2020      14          218.5    дополнительный
    7       01.04.2020      15          230.0    основной
    20      01.04.2020      16          209.0    дополнительный
    38      01.04.2020      17          123.5    основной
    18      01.04.2020      18          130.0    дополнительный
    22      01.04.2020      19          218.5    основной
    ...     ...             ...         ...         ...
    10      31.05.2021      848         117.0    основной

In [11]:
from datetime import datetime

cursor.execute("SELECT COUNT(*) FROM t")
count_t = cursor.fetchone()[0]

if count_t == 0:
    fake = Faker()

    # Заполнение таблицы t случайными данными
    for _ in range(10000):
        id_client = random.randint(1, 200)
        start_date = datetime(2021, 1, 1)
        end_date = datetime.now()
        tran_time = fake.date_between(start_date=start_date, end_date=end_date)
        id_tran = random.randint(1, 100000)
        sum_tran = round(random.uniform(10.0, 1000.0), 1)
        prod_name = random.choice(["основной", "дополнительный"])
        cursor.execute("INSERT INTO t (id_client, tran_time, id_tran, sum_tran, prod_name) VALUES (?, ?, ?, ?, ?)",
                       (id_client, tran_time, id_tran, sum_tran, prod_name))

    conn.commit()
else:
    print("Данные уже заполнены")
pd.read_sql("""
    SELECT * FROM t WHERE id_client = 1 ORDER BY 2
""", conn)

,id_client,tran_time,id_tran,sum_tran,prod_name
0,1,2021-01-13,12023,552.3,дополнительный
1,1,2021-01-30,18013,425.4,дополнительный
2,1,2021-02-10,51620,920.6,дополнительный
3,1,2021-03-17,58404,806.7,дополнительный
4,1,2021-03-24,16372,622.2,основной
5,1,2021-04-21,60927,373.2,основной
6,1,2021-05-22,8589,727.3,основной
7,1,2021-07-06,36489,859.3,основной
8,1,2021-07-17,94580,196.1,основной
9,1,2021-08-24,44669,919.5,основной


In [12]:
#cursor.execute('DELETE FROM t')
#conn.commit()

6. Посчитать траты каждого клиента за последний месяц скользящим окном.

In [13]:
data = pd.read_sql("SELECT * FROM t", conn)

data['tran_time'] = pd.to_datetime(data['tran_time'])
data = data.sort_values(by='tran_time') 
data = data.set_index('tran_time')

# Группирую данные по id_client и вычисляю скользящую сумму за последний месяц
rolling_sum = data.groupby('id_client')['sum_tran'].rolling('30D').sum().reset_index()

# Добавляю в исходную таблицу
rolling_month = data.merge(rolling_sum, on=['id_client', 'tran_time'], suffixes=('', '_rolling_sum'))
rolling_month = rolling_month.rename(columns={'sum_tran_rolling_sum': 'rolling_sum'})

rolling_month = rolling_month.sort_values(by=['id_client', 'tran_time'])

rolling_month.to_sql('rolling_month', conn, if_exists='replace', index=False)

print(rolling_month)

       id_client  tran_time  id_tran  sum_tran       prod_name  rolling_sum
110            1 2021-01-13    12023     552.3  дополнительный        552.3
243            1 2021-01-30    18013     425.4  дополнительный        977.7
358            1 2021-02-10    51620     920.6  дополнительный       1898.3
695            1 2021-03-17    58404     806.7  дополнительный        806.7
757            1 2021-03-24    16372     622.2        основной       1428.9
...          ...        ...      ...       ...             ...          ...
9429         200 2023-09-05    96042     108.8  дополнительный        619.2
9461         200 2023-09-09    93178     284.1  дополнительный        903.3
9517         200 2023-09-14    15314     164.7        основной       1068.0
9845         200 2023-10-19    18958     539.3        основной        539.3
10384        200 2023-12-13    24803     613.4        основной        613.4

[10428 rows x 6 columns]


7. Посчитать траты топ 100 клиентов по продукту ‘основной’.

In [14]:
pd.read_sql("""
    SELECT 
         id_client
        ,MAX(rolling_sum) as max_rolsum
    FROM rolling_month
    WHERE prod_name in ('основной')
    GROUP BY id_client
    ORDER BY MAX(rolling_sum) DESC
    LIMIT 100
""", conn)

,id_client,max_rolsum
0,180,4751.6
1,190,4697.1
2,24,4458.6
3,69,4424.5
4,128,4229.4
...,...,...
95,20,2804.7
96,67,2802.6
97,174,2799.4
98,63,2798.7


8. Что такое retention? Расскажите про разные виды retention

Retention (удержание) - метрика, показывающая эффективность удержания клиентов или пользователей, пользующихся тем или иным  продуктом/услугой. 

1) Повторяющееся удержание (rolling retention)
Повторяющееся удержание N-го дня показывает процент пользователей, которые вернулись в приложение в день N с момента установки или позже.
Оно учитывает вернувшимися в день N тех пользователей, которые зашли в приложение в день N или позже в любой другой день (N+m).

2) Полное удержание (full retention)
Полное удержание N-го дня показывает процент пользователей, которые заходили в приложение каждый день до дня N.

3) Возвратное удержание (return retention)
Возвратное удержание N-го дня показывает процент пользователей, которые вернулись хотя бы один раз за N дней.

4) Диапазонное удержание (bracket-dependent return retention)
Диапазонное удержание N-го дня является вариацией возвратного удержания. Оно фиксирует пользователей, вернувшихся в приложение в определенный период хотя бы один раз.
Для расчета этого удержания задается дополнительно к N параметр M, который ограничивает временной диапазон для возврата пользователей.
Удержание здесь рассчитывается как процент пользователей, вернувшихся в приложение в промежуток M-N дней.

9. Нужно посчитать retention по примеру ниже
0-текущий месяц, 1- Прошлый месяц и тд
 
     MIN_DATE                  0         1       2      3       4  
     2022-01-31 00:00:00    79 598    75 854  70 032  65 438 35 7322    
     2022-02-28 00:00:00    68 092    54 232  23 372   3 232 
     2022-03-31 00:00:00    70 504    52 031  32 232      
     2022-04-30 00:00:00    68 546    53 232          
     2023-03-31 00:00:00    10 332                                        

Я не очень понимаю, почему retention должен выглядеть так, как в примере, с учётом того, что вверху у нас самый первый месяц и относительно него не может быть предыдущего месяца, поэтому сделал зеркально

In [15]:
rolling_month['tran_time'] = pd.to_datetime(rolling_month['tran_time'])

rolling_month['MIN_DATE'] = rolling_month['tran_time'].dt.to_period('M')

# Уникальные клиенты за месяц
monthly_clients = rolling_month.groupby('MIN_DATE')['id_client'].nunique()

retention = pd.DataFrame(index=monthly_clients.index, columns=range(len(monthly_clients)))

for i in range(len(monthly_clients)):
    for j in range(i, len(monthly_clients)):
        retention.iloc[j, i] = rolling_month[rolling_month['MIN_DATE'] == monthly_clients.index[i]]['id_client'].isin(rolling_month[rolling_month['MIN_DATE'] == monthly_clients.index[j]]['id_client']).sum()

retention.index = retention.index.strftime('%Y-%m-%d')

print(retention)

             0    1    2    3    4    5    6    7    8    9   ...   26   27  \
MIN_DATE                                                      ...             
2021-01-31  260  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-02-28  188  277  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-03-31  198  223  292  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-04-30  201  213  227  314  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-05-31  192  195  207  242  278  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-06-30  194  210  226  224  203  299  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2021-07-31  208  202  236  249  216  208  316  NaN  NaN  NaN  ...  NaN  NaN   
2021-08-31  210  220  230  256  212  224  244  312  NaN  NaN  ...  NaN  NaN   
2021-09-30  203  214  228  245  222  232  247  257  280  NaN  ...  NaN  NaN   
2021-10-31  187  195  206  221  190  215  230  229  201  279  ...  NaN  NaN   
2021-11-30  186  200  195  210  196  219  221  219  

10. Нужно посчитать retention по примеру ниже, и вывести доли

     MIN_DATE               0        1       2     3       4      5      6  
     2022-01-31 00:00:00    79598    54%    47%    42%    41%    38%    35%   
     2022-02-28 00:00:00    68092    54%    45%    42%    40%    37%    36%   
     2022-03-31 00:00:00    70504    52%    45%    41%    38%    36%    34%   
     2022-04-30 00:00:00    68546    56%    48%    43%    41%    38%    35%    
     2022-05-31 00:00:00    73996    56%    48%    45%    42%    39%    37%   
     2022-06-30 00:00:00    82400    56%    49%    45%    42%    39%    36%    
     2022-07-31 00:00:00    85074    59%    51%    46%    44%    41%    38%            
     2022-08-31 00:00:00    97628    58%    50%    47%    44%    40%    37%                   
     2022-09-30 00:00:00    90656    56%    49%    46%    42%    39%    37%                    
     2022-10-31 00:00:00    93869    57%    50%    45%    42%    40%                        
     2022-11-30 00:00:00    99177    55%    48%    45%    43%                            
     2022-12-31 00:00:00    10490    55%    50%    45%                                
     2023-01-31 00:00:00    96691    55%    49%                                    
     2023-02-28 00:00:00    95876    56%                                        
     2023-03-31 00:00:00    10332 

Аналогично комментарию про предыдущий retention

In [16]:
rolling_month['tran_time'] = pd.to_datetime(rolling_month['tran_time'])

rolling_month['MIN_DATE'] = rolling_month['tran_time'].dt.to_period('M')

# Уникальные клиенты за месяц
monthly_clients = rolling_month.groupby('MIN_DATE')['id_client'].nunique()

retention = pd.DataFrame(index=monthly_clients.index, columns=range(len(monthly_clients)))
total_customers = monthly_clients.iloc[0]

for i in range(len(monthly_clients)):
    current_month_customers = monthly_clients.iloc[i]
    retention.iloc[i, 0] = current_month_customers
    for j in range(1, i+1):
        retention.iloc[i, j] = round(retention.iloc[i, j-1] / total_customers * 100, 2)

retention.index = retention.index.strftime('%Y-%m-%d')
retention.columns = range(len(monthly_clients))

print(retention)

             0       1      2      3      4      5      6      7     8     9   \
MIN_DATE                                                                        
2021-01-31  148     NaN    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   
2021-02-28  148   100.0    NaN    NaN    NaN    NaN    NaN    NaN   NaN   NaN   
2021-03-31  153  103.38  69.85    NaN    NaN    NaN    NaN    NaN   NaN   NaN   
2021-04-30  155  104.73  70.76  47.81    NaN    NaN    NaN    NaN   NaN   NaN   
2021-05-31  143   96.62  65.28  44.11   29.8    NaN    NaN    NaN   NaN   NaN   
2021-06-30  150  101.35  68.48  46.27  31.26  21.12    NaN    NaN   NaN   NaN   
2021-07-31  154  104.05   70.3   47.5  32.09  21.68  14.65    NaN   NaN   NaN   
2021-08-31  156  105.41  71.22  48.12  32.51  21.97  14.84  10.03   NaN   NaN   
2021-09-30  158  106.76  72.14  48.74  32.93  22.25  15.03  10.16  6.86   NaN   
2021-10-31  143   96.62  65.28  44.11   29.8  20.14  13.61    9.2  6.22   4.2   
2021-11-30  140   94.59  63.

In [17]:
conn.close()